In [ ]:
import pandas as pd
import numpy as np
features = pd.read_csv('/Users/johannessolibieda/Documents/GitHub/Its-Wekk/Fucking Final Data/Working_DataFrame.csv')
target = pd.read_csv('/Users/johannessolibieda/Documents/GitHub/Its-Wekk/Fucking Final Data/CleanedFinal_Target_Data_Combined_resid_Trend')

In [ ]:


# Define all features based on the columns of the features DataFrame
all_features = features.columns.tolist()

# Create a list to store the new features and their correlations
new_feature_correlations = []

# Create a set to keep track of added features
added_features = set()

# Dictionary to store new features
new_features_dict = {}

# Iterate over each pair of features to create new features
for i in range(len(all_features)):
    for j in range(i, len(all_features)):
        feature1 = all_features[i]
        feature2 = all_features[j]
        
        # Skip the 'Datum' column
        if feature1 == 'Datum' or feature2 == 'Datum':
            continue
        
        # Create new features by multiplying and calculating growth
        new_feature_name = f'{feature1}_x_{feature2}'
        new_feature_growth_name = f'{feature1}_growth'
        
        # Multiply features
        new_features_dict[new_feature_name] = features[feature1] * features[feature2]
        
        # Calculate growth (difference to last period)
        new_features_dict[new_feature_growth_name] = features[feature1].diff()

        # Calculate growth (difference to last period) of the multiplied features
        new_feature_mult_growth_name = f'{new_feature_name}_growth'
        new_features_dict[new_feature_mult_growth_name] = new_features_dict[new_feature_name].diff()
        
        # Calculate correlations with the target variable
        correlation_mult = new_features_dict[new_feature_name].corr(target['PM10_Combined_Trend_Residual'])
        correlation_growth = new_features_dict[new_feature_growth_name].corr(target['PM10_Combined_Trend_Residual'])
         
        # Store the correlations if they are high and not already added
        if abs(correlation_mult) > 0.1 and new_feature_name not in added_features:
            new_feature_correlations.append((new_feature_name, correlation_mult))
            added_features.add(new_feature_name)
        if abs(correlation_growth) > 0.1 and new_feature_growth_name not in added_features:
            new_feature_correlations.append((new_feature_growth_name, correlation_growth))
            added_features.add(new_feature_growth_name)

# Concatenate new features to the original DataFrame
features = pd.concat([features, pd.DataFrame(new_features_dict)], axis=1)

# Print the new features with high correlations
for feature, corr in new_feature_correlations:
    print(f'Feature: {feature}, Correlation: {corr}')


Feature: Basel Temperature [2 m elevation corrected]_x_Basel Temperature [2 m elevation corrected], Correlation: 0.295880312664073
Feature: Basel Temperature [2 m elevation corrected]_x_Basel Wind Speed [10 m], Correlation: 0.1649281135229291
Feature: Basel Temperature [2 m elevation corrected]_x_660n_sumPW, Correlation: 0.11947271525695315
Feature: Basel Temperature [2 m elevation corrected]_x_Gasverbrauch, Correlation: -0.33163000210476157
Feature: Basel Temperature [2 m elevation corrected]_x_Traffic, Correlation: 0.10683501722873956
Feature: Basel Precipitation Total_x_Hour, Correlation: -0.133496831045994
Feature: Basel Wind Speed [10 m]_x_Basel Wind Speed [10 m], Correlation: -0.13793937770227843
Feature: Basel Wind Speed [10 m]_x_Basel Wind Direction [10 m], Correlation: -0.10142799072307421
Feature: Basel Wind Speed [10 m]_x_Gasverbrauch, Correlation: -0.16278068991896746
Feature: Basel Wind Speed [10 m]_x_Hour, Correlation: -0.16034332300631474
Feature: Basel Wind Direction [1

In [24]:
MostImp = pd.read_csv('/Users/johannessolibieda/Documents/GitHub/Its-Wekk/5 - Models/4 - LinReg/0 - ModelData/50MostImpFeatures_DF.csv')

In [ ]:

# Example DataFrame (replace with your actual DataFrame)
df = features
dfa = pd.DataFrame()
# Define feature pairs and their names
feature_pairs = [
    ("Basel Temperature [2 m elevation corrected]", "Basel Temperature [2 m elevation corrected]"),
    ("Basel Temperature [2 m elevation corrected]", "Basel Wind Speed [10 m]"),
    ("Basel Temperature [2 m elevation corrected]", "660n_sumPW"),
    ("Basel Temperature [2 m elevation corrected]", "Gasverbrauch"),
    ("Basel Temperature [2 m elevation corrected]", "Traffic"),
    ("Basel Precipitation Total", "Hour"),
    ("Basel Wind Speed [10 m]", "Basel Wind Speed [10 m]"),
    ("Basel Wind Speed [10 m]", "Basel Wind Direction [10 m]"),
    ("Basel Wind Speed [10 m]", "Gasverbrauch"),
    ("Basel Wind Speed [10 m]", "Hour"),
    ("Basel Wind Direction [10 m]", "Basel Wind Direction [10 m]"),
    ("Basel Wind Direction [10 m]", "Hour"),
    ("406v_sumLW", "408n_sumLief"),
    ("660n_sumPW", "Gasverbrauch"),
    ("Gasverbrauch", "Gasverbrauch"),
    ("Gasverbrauch", "Hour"),
]

# Loop through the pairs, calculate the product, and add to the DataFrame
for feature1, feature2 in feature_pairs:
    new_feature_name = f"{feature1}_x_{feature2}"
    dfa[new_feature_name] = df[feature1] * df[feature2]

# Save or view the updated DataFrame
# df.to_csv('updated_data.csv', index=False)
print(dfa.head())

print(f'Size of dfa: {dfa.shape}')


   Basel Temperature [2 m elevation corrected]_x_Basel Temperature [2 m elevation corrected]  \
0                                           0.858340                                           
1                                           1.085695                                           
2                                           1.351255                                           
3                                           1.288480                                           
4                                           1.354144                                           

   Basel Temperature [2 m elevation corrected]_x_Basel Wind Speed [10 m]  \
0                                           0.643029                       
1                                           0.956551                       
2                                           0.996992                       
3                                           0.943412                       
4                                          

In [29]:
from sklearn.preprocessing import StandardScaler

# Concatenate dfa to MostImp
MostImp_combined = pd.concat([MostImp, dfa], axis=1)

# Standardize the features
scaler = StandardScaler()
MostImp_combined_scaled = pd.DataFrame(scaler.fit_transform(MostImp_combined.iloc[:, 1:]), columns=MostImp_combined.columns[1:])
MostImp_combined_scaled.insert(0, 'Datum', MostImp_combined['Datum'])

# Export the combined DataFrame to a CSV file
MostImp_combined_scaled.to_csv('50MostImp+CombinedFeatures.csv', index=False)